#### We need to install the ktrain library. Its a light weight wrapper for keras to help train neural networks. With only a few lines of code it allows you to build models, estimate optimal learning rate, loading and preprocessing text and image data from various sources and much more. More about our approach can be found at [this](https://towardsdatascience.com/bert-text-classification-in-3-lines-of-code-using-keras-264db7e7a358) article.

In [1]:
!pip3 install ktrain==0.26.3
# use tensorflow 2.4.0

  Using cached seqeval-0.0.19-py3-none-any.whl


  Attempting uninstall: seqeval
    Found existing installation: seqeval 0.0.12
    Uninstalling seqeval-0.0.12:
      Successfully uninstalled seqeval-0.0.12


In [2]:
#Importing
import ktrain
from ktrain import text

In [3]:
##obtain the dataset
import os
try :
    from google.colab import files
    import tensorflow as tf
    dataset = tf.keras.utils.get_file(
        fname="aclImdb.tar.gz", 
        origin="http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz", 
        extract=True,
    )
    IMDB_DATADIR = os.path.join(os.path.dirname(dataset), "aclImdb")
except ModuleNotFoundError :
    if not os.path.exists(os.getcwd()+"\\Data\\aclImdb") :
        import tensorflow as tf
        dataset = tf.keras.utils.get_file(
            fname="aclImdb.tar.gz", 
            origin="http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz", 
            extract=True,
        )

        # set path to dataset
        IMDB_DATADIR=os.getcwd()
    else :

        # set path to dataset
        IMDB_DATADIR=os.getcwd()+"\\Data\\aclImdb"

84131840/84125825 [==============================] - 4s 0us/step


## STEP 1: Preprocessing
####The texts_from_folder function will load the training and validation data from the specified folder and automatically preprocess it according to BERT's requirements. In doing so, the BERT model and vocabulary will be automatically downloaded.

In [6]:
(x_train, y_train), (x_test, y_test), preproc = text.texts_from_folder(IMDB_DATADIR, 
                                                                       maxlen=500, 
                                                                       preprocess_mode='bert',
                                                                       train_test_names=['train', 
                                                                                         'test'],
                                                                       classes=['pos', 'neg'])

downloading pretrained BERT model and vocabulary...
[██████████████████████████████████████████████████]
extracting pretrained BERT model and vocabulary...
done.

cleanup downloaded zip...
done.

preprocessing train...


preprocessing test...


### STEP 2: Loading a pre trained BERT and wrapping it in a ktrain.learner object

In [7]:
model = text.text_classifier('bert', (x_train, y_train), preproc=preproc)
learner = ktrain.get_learner(model,train_data=(x_train, y_train), val_data=(x_test, y_test), batch_size=6)

Is Multi-Label? False
maxlen is 500




Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.









done.


### STEP 3: Training and Tuning the model's parameters

In [8]:
learner.fit_onecycle(2e-5, 4)



begin training using onecycle policy with max lr of 2e-05...
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 25000 samples, validate on 25000 samples
Epoch 1/4
25000/25000 [==============================] - 4557s 182ms/step - loss: 0.2551 - acc: 0.8944 - val_loss: 0.1694 - val_acc: 0.9342
Epoch 2/4
25000/25000 [==============================] - 4545s 182ms/step - loss: 0.1607 - acc: 0.9419 - val_loss: 0.1692 - val_acc: 0.9362
Epoch 3/4
25000/25000 [==============================] - 4545s 182ms/step - loss: 0.0875 - acc: 0.9706 - val_loss: 0.1860 - val_acc: 0.9378
Epoch 4/4
25000/25000 [==============================] - 4543s 182ms/step - loss: 0.0242 - acc: 0.9932 - val_loss: 0.2394 - val_acc: 0.9385
